my_strategy() with:
- n depth recursion
- alpa-beta pruned search

Typical losses / 1000:
- depth 0: 5.0% in 4 seconds
- depth 1: 12.2%, 11.1%, 11.1% in 43 seconds
- depth 2: 2.5% in 2.5 mins 
- depth 3: 3.4% in 10 mins
- depth 4: 1.6% in 36 mins
- depth 5: ?? in 140 mins
- depth 6: ?? in 550 mins

Pruning:
- does not seem to have harmed accuracy
- improves speed sigificantly at greater depths: x3 depth 3, x5 depth 4

Odd depths: still worse than even depths (not surprising, same algorithm)


In [14]:
# CONNECT 4 - V4 - RECURSIVE WITH PUNING

import numpy as np

# CONSTANTS

# Game control
DEPTH = 0
DEBUG_OUTPUT = False
# Game definition
ROWS = 6
COLUMNS = 7
CONNECT_N = 4
# Player definition - values ascribed later
AI_PLAYER = None
HUMAN_PLAYER = None
# Board and player colours
COLOUR_EMPTY = "\u2022 "
COLOUR_RED = "\U0001F534"
COLOUR_YELLOW = "\U0001F7E1"
COLOUR_BLACK = "\u26AB" # black just fills the space for testing
COLOUR_COUNTER=[COLOUR_EMPTY, COLOUR_RED, COLOUR_YELLOW, COLOUR_BLACK]
COLOUR_TEXT = ["Blank", "Red", "Yellow", "Black"]

def my_strategy(board, player_num):
    # To interface into tester
    # converts list of lists [[int]*6]*7 into np array
    global AI_PLAYER, HUMAN_PLAYER
    AI_PLAYER = player_num
    HUMAN_PLAYER = AI_PLAYER ^ 3
    return get_ai_move(np.array(board))

def print_debug(str):
    # Print debug output if enabled
    if DEBUG_OUTPUT:
        print(str)

def flip_coin():
    # Returns True 50% of the time
    return np.random.randint(2) == 1
    
def get_valid_moves(board):
    # Returns a list of valid moves for the current board state
    valid_moves = []
    for col in range(COLUMNS):
        if board[col,ROWS-1] == 0:
            valid_moves.append(col)
    return valid_moves

def check_winner(board, player):
    # Check horizontal, vertical, and diagonal lines for a win
    for col in range(COLUMNS):
        for row in range(ROWS):
            # If cell is empty, no need to check this cell or cells above
            if board[col, row] == 0:
                break
            # If cell is not the player, skip but keep checking cells above
            if board[col, row] != player:
                continue
            # If room above... check vertical
            if (row + CONNECT_N <= ROWS):
                if all (board[col, row + i] == player for i in range(1, CONNECT_N)):
                    return True
            # If room to the right...
            if (col + CONNECT_N <= COLUMNS):  # if room to the right
                # ...check horizontal-right
                if all (board[col + i, row] == player for i in range(1, CONNECT_N)):
                    return True
                # ...check diagonal up-right if room above
                if (row + CONNECT_N <= ROWS):
                    if all (board[col + i, row + i] == player for i in range(1, CONNECT_N)):
                        return True
                # ...check diagonal down-right if room below
                if (row - CONNECT_N >= -1):
                    if all (board[col + i, row - i] == player for i in range(1, CONNECT_N)):
                        return True
    return False

def check_draw(board):
    return all (board[column, ROWS-1] !=0 for column in range(COLUMNS))

def is_terminal_node(board):
    # Check for Terminal Node (Win, Draw, or Continue)
    return check_winner(board, AI_PLAYER) or check_winner(board, HUMAN_PLAYER) or check_draw(board)

def evaluate_board(board, col, row):

    def eval_get_counts(board, player_num, col, row, col_step, row_step):
    # Returns the number of adjacent, separate, and blank cells in a line

        def is_in_range(x, max):
            return x >= 0 and x < max

        count_same_adj = 0
        count_same_sep = 0
        count_blank = 0

        # evaluate in both directions along the line dir -1 and 1
        for eval_dir in range(-1,2,2):
            check_column = col + col_step * eval_dir
            check_row = row + row_step * eval_dir
            check_adjacent = True
            # explore up to CONNECT_N-1 spaces either side
            for depth in range(CONNECT_N - 1):
                # stop if off the board
                if not (is_in_range(check_column,7) and is_in_range(check_row,6)):
                    break
                # stop if cell is opponent's
                if board[check_column,check_row] == player_num ^ 3:
                    break
                # if the cell is a blank, count it but set adjacent to false
                if board[check_column,check_row] == 0:
                    count_blank += 1
                    check_adjacent = False
                # if the cell is the same colour, count adjacent or separate
                if board[check_column,check_row] == player_num:
                    if check_adjacent:
                        count_same_adj += 1
                    else:
                        count_same_sep += 1
                check_column += col_step * eval_dir
                check_row += row_step * eval_dir
        return count_same_adj, count_same_sep, count_blank

    def eval_line(board, col, row, col_step, row_step):
    # Returns a line score based on the number of adjacent, separate, and blank cells
    # Score is from the perspective of last player i.e. player_num in [col,row]

        # evaluation weightings
        EVAL_SAME_ADJ = 2 # exponential
        EVAL_SAME_SEP = 1 # linear
        EVAL_BLANK = 1 # linear

        eval_line_score = 0
        player_num = board[col,row]

        # Calculate offensive score
        count_same_adj, count_same_sep, count_blank = eval_get_counts(board, player_num, col, row, col_step, row_step)
        
        # Only score a line if there is enough space to win
        # NB function might return eval_line_score=0 (better than losing move)
        if (count_same_adj + count_same_sep + count_blank) >= CONNECT_N - 1:
            if count_same_adj > 0:
                eval_line_score += EVAL_SAME_ADJ ** count_same_adj
            eval_line_score += count_same_sep * EVAL_SAME_SEP
            eval_line_score += count_blank * EVAL_BLANK
            print_debug(f"offense: count_same_adj: {count_same_adj}, count_same_sep: {count_same_sep}, count_blank: {count_blank}, eval score: {eval_line_score}")

        # Add a defensive score
        count_same_adj, count_same_sep, count_blank = eval_get_counts(board, player_num ^ 3, col, row, col_step, row_step)
        
        # Only score a line if there is enough space for opponent to win
        if (count_same_adj + count_same_sep + count_blank) >= CONNECT_N - 1:
            if count_same_adj > 0:
                eval_line_score += EVAL_SAME_ADJ ** count_same_adj
            eval_line_score += count_same_sep * EVAL_SAME_SEP
            eval_line_score += count_blank * EVAL_BLANK
            print_debug(f"defense: count_same_adj: {count_same_adj}, count_same_sep: {count_same_sep}, count_blank: {count_blank}, eval score: {eval_line_score}")

        return eval_line_score

    def eval_move(board, col, row):
    # Returns a score for a move based on the evaluation of each line

        eval_move_score = 0
        eval_steps = np.array([[1, 0], [1, 1], [0, 1], [1, -1]])
        # Evaluate each line in turn
        for i in range(4):
            print_debug(f"evaluating line {i}")
            eval_move_score += eval_line(board, col, row, eval_steps[i, 0], eval_steps[i, 1])
        print_debug(f"column {col} score: {eval_move_score}")
        return eval_move_score

    # Check for a win, loss, or draw, otherwise return a heuristic evaluation
    if check_winner(board, AI_PLAYER):
        return np.inf  # AI wins
    elif check_winner(board, HUMAN_PLAYER):
        return -np.inf  # Human wins
    elif check_draw(board):
        return 0  # Draw
    else:
        # Heuristic evaluation for intermediate states (foccsing on the move that led to this board state)
        return eval_move(board, col, row)

def minimax(board, col, row, depth, maximizing_player, alpha=-np.inf, beta=np.inf):
    """
    Perform the minimax algorithm with alpha-beta pruning to determine the best move for a player in a two-player game.
    Args:
        board (np.ndarray): The current state of the game board.
        col (int): The column index of the last move made.
        row (int): The row index of the last move made.
        depth (int): The maximum depth to search in the game tree.
        maximizing_player (bool): True if the current player is the maximizing player (AI), False if the current player is the minimizing player (Human).
        alpha (float): The best value that the maximizer currently can guarantee at that level or above.
        beta (float): The best value that the minimizer currently can guarantee at that level or above.
    Returns:
        int: The best score for the maximizing player at the top level of the game tree.
    Summary:
        - This function uses a recursive approach to explore all possible moves up to a given depth.
        - This is a generic gaming function that can be used for any two-player game
        - Each level represents a player's turn, alternating between maximizing and minimizing players
    Notes:
        - The maximizing player aims to maximize the score, while the minimizing player aims to minimize it.
        - The function terminates when the depth is 0 or a terminal node (win/loss/draw) is reached.
        - The function evaluates the board state using the `evaluate_board` function and determines valid moves using the `get_valid_moves` function.
        - The `make_move` function is used to simulate moves on the board.
    More on the Return value:
        - The return value is the best score that the maximizing player can achieve from the current board state, considering all possible moves up to the specified depth.
        - This score is determined by recursively evaluating the game tree and propagating the best scores back up the recursion chain.
        - The top-level return value represents the best score for the maximizing player (AI) at the root node of the game tree.
    More on the algorithm used:
        - Alpha-beta pruning optimizes the search by cutting off branches that do not need to be explored.
    Why ODD Depth values:
        - The algorithm tends to win more games with odd depth values because it allows the AI (maximizing player) to make the final move in the search tree.
        - With an odd depth, the AI evaluates the board state after the human (minimizing player) has made their move, giving the AI the opportunity to respond optimally.
        - This ensures that the AI's strategy is based on the most recent move by the opponent, leading to better decision-making and higher chances of winning.
    """

    if depth == 0 or is_terminal_node(board):
        return evaluate_board(board, col, row)
    
    valid_moves = get_valid_moves(board)
    if maximizing_player:
        value = -np.inf
        for col in valid_moves:
            new_board = board.copy()
            row = make_move(new_board, col, AI_PLAYER)  # AI is the maximizing player
            value = max(value, minimax(new_board, col, row, depth-1, False, alpha, beta))
            alpha = max(alpha, value)
            if alpha >= beta:
                break  # Beta cut-off
        if depth == DEPTH:
            print_debug(f"AI value (best case): {value}")
        return value
    else:
        value = np.inf
        for col in valid_moves:
            new_board = board.copy()
            row = make_move(new_board, col, HUMAN_PLAYER)  # Human is the minimizing player
            value = min(value, minimax(new_board, col, row, depth-1, True, alpha, beta))
            beta = min(beta, value)
            if alpha >= beta:
                break  # Alpha cut-off
        if depth == DEPTH:
            print_debug(f"Human value (worst case): {value}")
        return value

def make_move(board, col, player):
    # Updates the board col with the player's move amd return the row
    row = next(r for r in range(ROWS) if board[col, r] == 0)
    board[col, row] = player
    return row

def get_ai_move(board):
    # Returns the best move for the AI player
    valid_moves = get_valid_moves(board)
    best_score = -np.inf
    best_col = valid_moves[0]  # Default - just in case the only moves are losing
    for col in valid_moves:
        new_board = board.copy()
        row = make_move(new_board, col, AI_PLAYER)
        # This initiates the minimax recursion
        score = minimax(new_board, col, row, DEPTH, False)
        # Choose the best move
        if score > best_score or (score == best_score and flip_coin()):
            best_score = score
            best_col = col
    return best_col

In [22]:
# When you're ready to run your strategy run the top cell, then this cell
# You can do this as often as you like as you improve your strategy
from assessment.assessor import assess

assess(my_strategy, 1000)

Assessing student strategy...
                                                                                                    
Results
Wins: 879
Draws: 0
Losses: 121
Forfeits: 0
Mark: 75.94%
